In [3]:
import openai
import os
import pandas as pd
import time
from sklearn.metrics import *
import nltk
import warnings
import numpy as np
from itertools import chain


from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

# store api in text file, extract here
with open("api.txt", 'r') as f:
    api = f.read()

# openai.api_key  = os.getenv(api)
openai.api_key = api

## ChatGPT event phrase extraction & date classification

In [7]:
# load ChatGPT run function for date classification & event phrase extraction
from scripts.chatgpt_extraction import run

# load training data
gt_train = pd.read_csv("data/GT/GTtrain/date_event_combinations.csv")
gt_train['class'] = gt_train['label'].apply(lambda x: x.split(': ')[1] if ': ' in x else x)

# load test data -> all dates except the decision dates
df_test = pd.read_csv("data/results/chatgpt_extraction/input.csv")
run(df_test, 1, "data/results/chatgpt_extraction/", gt_train)


## ChatGPT event classification

In [8]:
# first we need to select all dates with an event from all selected dates (except decision dates)
GT_alldates = pd.read_csv("data/GT/GTtest/all_dates.csv")

from scripts.remove_mistakes import remove_no_event_dates
test_event_dates = remove_no_event_dates(df_test, GT_alldates)
test_event_dates.to_csv("data/results/chatgpt_classification/input.csv", index = False)

Original length of dataframe:5
New length after removing mistakes: 2


In [9]:
# load ChatGPT run function for event classification
from scripts.chatgpt_classification import run

# use same training data as chatgpt_extraction
run(test_event_dates, 1, "data/results/chatgpt_classification/", gt_train)